# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [153]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score, make_scorer
from scipy.stats import gmean
import re
import pickle
import numpy as np

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /Users/Utyubayev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Utyubayev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Utyubayev/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [154]:
# load data from database
engine = create_engine('sqlite:///Disaster_response.db')
df = pd.read_sql_table('Messages', engine)

In [155]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [156]:
X = df['message']
Y = df[df.columns.to_list()[4:]]
print(X.shape)
print(Y.shape)

(26039,)
(26039, 35)


In [157]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [158]:
X.isnull().sum()

0

In [159]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
Y.isnull().sum().any()

False

### 2. Write a tokenization function to process your text data

In [161]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

## Start with LogisticRegression with class_weight='balanced' as classes are imbalanced

In [162]:
pipeline = Pipeline([
    ('vector', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LogisticRegression(class_weight='balanced', n_jobs=-1),n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vector',
                 CountVectorizer(tokenizer=<function tokenize at 0x1a65344b00>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                                    n_jobs=-1),
                                       n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [164]:
# for each category of labels (each column of Y) 
    #   1 print out classification_report
    #   2 compute f1 score for the positive class (value = 1)
    # then compute an average of all the f1 scores (global_f1_score)
def check_results(y_test, y_pred, to_print=False):
    f1_array = []
    for index, column in enumerate(y_test.columns):
        prediction = y_pred[:,index]
        actual = y_test.iloc[:,index]
        if to_print:
            print('Category = {}'.format(column))
            print(classification_report(actual, prediction))
            #print(precision_recall_fscore_support(actual, prediction, average='binary'))
        f1 = f1_score(actual, prediction)
        f1_array.append(f1)
    global_f1_score = np.mean(f1_array)
    if to_print:
        print('Arithmetic mean of f1 scores for each category = {}'.format(round(global_f1_score,2)))
    return global_f1_score

## Created a custom scorer, but it turned out to be the same as 'f1_macro' scorer option 

In [165]:
# create a custom scorer function based on the above 
f1_custom_scorer = make_scorer(check_results, to_print=False)

In [166]:
y_pred = pipeline.predict(X_test)

In [167]:
check_results(y_test, y_pred, True)

Category = related
              precision    recall  f1-score   support

         0.0       0.31      0.45      0.37      1268
         1.0       0.79      0.68      0.73      3940

    accuracy                           0.63      5208
   macro avg       0.55      0.57      0.55      5208
weighted avg       0.68      0.63      0.64      5208

Category = request
              precision    recall  f1-score   support

         0.0       0.92      0.75      0.83      4318
         1.0       0.36      0.68      0.47       890

    accuracy                           0.74      5208
   macro avg       0.64      0.72      0.65      5208
weighted avg       0.82      0.74      0.77      5208

Category = offer
              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00      5193
         1.0       0.03      0.07      0.04        15

    accuracy                           0.99      5208
   macro avg       0.51      0.53      0.52      5208
weighted avg       

              precision    recall  f1-score   support

         0.0       0.94      0.78      0.85      4800
         1.0       0.14      0.40      0.20       408

    accuracy                           0.75      5208
   macro avg       0.54      0.59      0.53      5208
weighted avg       0.88      0.75      0.80      5208

Category = storm
              precision    recall  f1-score   support

         0.0       0.93      0.83      0.88      4724
         1.0       0.20      0.41      0.27       484

    accuracy                           0.79      5208
   macro avg       0.57      0.62      0.57      5208
weighted avg       0.86      0.79      0.82      5208

Category = fire
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      5155
         1.0       0.01      0.02      0.01        53

    accuracy                           0.96      5208
   macro avg       0.50      0.50      0.50      5208
weighted avg       0.98      0.96      0.

0.15770741439208227

## The average f1 score for basic Logistic Regression is only 0.15 due to imbalanced classes 

### 6. Improve your model
Use grid search to find better parameters. 

In [168]:
pipeline.get_params()

{'memory': None,
 'steps': [('vector',
   CountVectorizer(tokenizer=<function tokenize at 0x1a65344b00>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                      n_jobs=-1),
                         n_jobs=-1))],
 'verbose': False,
 'vector': CountVectorizer(tokenizer=<function tokenize at 0x1a65344b00>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                    n_jobs=-1),
                       n_jobs=-1),
 'vector__analyzer': 'word',
 'vector__binary': False,
 'vector__decode_error': 'strict',
 'vector__dtype': numpy.int64,
 'vector__encoding': 'utf-8',
 'vector__input': 'content',
 'vector__lowercase': True,
 'vector__max_df': 1.0,
 'vector__max_features': None,
 'vector__min_df': 1,
 'vector__ngram_range': (1, 1),
 'vector__preprocessor': None,
 'vector__s

In [169]:
parameters = {
    #'vector__max_df' : [1],
    'vector__ngram_range': [(1,1), (1,2)]
    #'tfidf__use_idf': [True, False],
    #'vector__stop_words': [None, 'english'],
    #'clf__estimator__learning_rate': [0.5, 1],
    #'clf__estimator__n_estimators': [50, 75],
    #'clf__estimator__max_iter': [100],
    #'clf__estimator__penalty': ['l1','l2'],
    #'clf__estimator__tol': [0.0001]
}

cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs=-1, cv=3, scoring='f1_macro')
cv.fit(X_train, y_train)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
print(cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)
check_results(y_test, y_pred, True)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

## Trying AdaBoost 

In [ ]:
pipeline_ada = Pipeline([
    ('vector', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(
                    base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced')
                )
            )
    )
])

In [ ]:
pipeline_ada.fit(X_train, y_train)

In [ ]:
y_pred_ada = pipeline_ada.predict(X_test)

In [ ]:
check_results(y_test, y_pred_ada, True)

## The results for AdaBoost are better than for the Logistic Regression 

In [ ]:
pipeline_ada.get_params()

In [ ]:
parameters = {
    #'vector__max_df' : [0.5, 1],
    #'vector__ngram_range': [(1,1), (1,2)],
    #'tfidf__use_idf': [True, False],
    #'vector__stop_words': [None, 'english'],
    'clf__estimator__learning_rate': [0.5, 1]
    #'clf__estimator__n_estimators': [50, 75],
    #'clf__estimator__base_estimator__max_depth': [1,2]
    #'clf__estimator__max_iter': [100],
    #'clf__estimator__penalty': ['l1','l2'],
    #'clf__estimator__tol': [0.0001]
}

cv_ada_2 = GridSearchCV(pipeline_ada, param_grid = parameters, n_jobs=-1, cv=3, scoring='f1_macro')
cv_ada_2.fit(X_train, y_train)

In [ ]:
print(cv_ada_2.best_params_)
print(cv_ada_2.best_score_)

In [ ]:
print(cv_ada_2.best_estimator_)

In [ ]:
y_pred_cv_ada  = cv_ada_2.predict(X_test)
check_results(y_test, y_pred_cv_ada, True)

In [ ]:
# checked to see if custom scorer and f1_macro is the same thing
f1_score(y_test, y_pred_cv_ada, average='macro')

## This step did not produce any improvement 

In [ ]:
parameters = {
    #'vector__max_df' : [0.5, 1],
    #'vector__ngram_range': [(1,1), (1,2)],
    #'tfidf__use_idf': [True, False],
    #'vector__stop_words': [None, 'english'],
    #'clf__estimator__learning_rate': [0.5, 1],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__base_estimator__max_depth': [1,2]
    #'clf__estimator__max_iter': [100],
    #'clf__estimator__penalty': ['l1','l2'],
    #'clf__estimator__tol': [0.0001]
}

#cv_ada_3 = GridSearchCV(cv_ada_2.best_estimator_, param_grid = parameters, n_jobs=-1, cv=3, scoring=f1_custom_scorer)
#cv_ada_3.fit(X_train, y_train)

In [ ]:
#print(cv_ada_3.best_params_)

In [ ]:
#y_pred_cv_ada  = cv_ada_3.predict(X_test)
#check_results(y_test, y_pred_cv_ada)

### 9. Export your model as a pickle file

In [ ]:
model = cv_ada_2.best_estimator_
filename = 'ava_model_1.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.